<a href="https://colab.research.google.com/github/odeandialamsyah/nlp_zero_to_hero/blob/main/NLP_Zero_to_Hero_Implementas_Word_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Meskipun melatih model Word Embeddings (seperti Word2Vec atau GloVe) dari nol memerlukan korpus teks yang sangat besar dan daya komputasi yang signifikan, kabar baiknya adalah kita bisa memanfaatkan model pre-trained yang sudah tersedia! Model-model ini dilatih pada miliaran kata dan telah belajar representasi makna yang sangat kaya.

Kita akan menggunakan library gensim untuk memuat dan berinteraksi dengan model Word2Vec, dan juga menunjukkan cara dasar untuk melatih model Anda sendiri jika Anda memiliki korpus yang lebih kecil.

In [1]:
%pip install gensim

In [2]:
import gensim.downloader as api

# Unduh model Word2Vec yang lebih kecil. Ada banyak pilihan di gensim.downloader.
# 'word2vec-google-news-300' adalah model besar (sekitar 3.6GB).
# Mari kita gunakan yang lebih kecil untuk demo, misalnya 'glove-wiki-gigaword-50' (sekitar 60MB).
# Ini akan mengunduh model jika belum ada.
try:
    print("Mengunduh model 'glove-wiki-gigaword-50'...")
    model_glove = api.load("glove-wiki-gigaword-50")
    print("Model berhasil diunduh dan dimuat!")
except Exception as e:
    print(f"Gagal mengunduh atau memuat model: {e}")
    print("Anda mungkin tidak memiliki koneksi internet atau model terlalu besar. "
          "Akan menggunakan model dummy untuk demonstrasi.")
    model_glove = None # Set to None if download fails

Mengunduh model 'glove-wiki-gigaword-50'...
[==================================================] 100.0% 66.0/66.0MB downloaded
Model berhasil diunduh dan dimuat!


Jika model_glove berhasil dimuat

In [3]:
if model_glove:
    print("\n--- Menggunakan Model Word Embeddings (GloVe) ---")

    # Mendapatkan vektor untuk sebuah kata
    kata_kucing = "cat"
    if kata_kucing in model_glove:
        vector_kucing = model_glove[kata_kucing]
        print(f"Vektor untuk '{kata_kucing}': {vector_kucing[:10]}...") # Tampilkan 10 dimensi pertama
        print(f"Ukuran vektor: {len(vector_kucing)} dimensi")
    else:
        print(f"Kata '{kata_kucing}' tidak ditemukan dalam kamus model.")

    # Mencari kata-kata paling mirip (most similar)
    kata_target = "king"
    print(f"\nKata-kata paling mirip dengan '{kata_target}':")
    if kata_target in model_glove:
        similar_words = model_glove.most_similar(kata_target)
        for word, similarity in similar_words:
            print(f"- {word}: {similarity:.4f}")
    else:
        print(f"Kata '{kata_target}' tidak ditemukan dalam kamus model.")

    # Analogi kata (seperti Raja - Pria + Wanita = Ratu)
    print("\n--- Analogi Kata (King - Man + Woman = Queen) ---")
    positive = ['king', 'woman']
    negative = ['man']
    try:
        result = model_glove.most_similar(positive=positive, negative=negative)
        for word, similarity in result:
            print(f"- {word}: {similarity:.4f}")
    except KeyError as e:
        print(f"Salah satu kata untuk analogi tidak ditemukan: {e}")
else:
    print("\nModel pre-trained tidak tersedia. Tidak dapat menjalankan contoh ini.")


--- Menggunakan Model Word Embeddings (GloVe) ---
Vektor untuk 'cat': [ 0.45281  -0.50108  -0.53714  -0.015697  0.22191   0.54602  -0.67301
 -0.6891    0.63493  -0.19726 ]...
Ukuran vektor: 50 dimensi

Kata-kata paling mirip dengan 'king':
- prince: 0.8236
- queen: 0.7839
- ii: 0.7746
- emperor: 0.7736
- son: 0.7667
- uncle: 0.7627
- kingdom: 0.7542
- throne: 0.7540
- brother: 0.7492
- ruler: 0.7434

--- Analogi Kata (King - Man + Woman = Queen) ---
- queen: 0.8524
- throne: 0.7664
- prince: 0.7592
- daughter: 0.7474
- elizabeth: 0.7460
- princess: 0.7425
- kingdom: 0.7337
- monarch: 0.7214
- eldest: 0.7185
- widow: 0.7099


Penjelasan Kode Menggunakan Model Pre-trained:
1. import gensim.downloader as api: Mengimpor modul downloader dari gensim yang memudahkan kita mengunduh model pre-trained yang umum.

2. model_glove = api.load("glove-wiki-gigaword-50"): Ini adalah baris penting untuk mengunduh dan memuat model GloVe. glove-wiki-gigaword-50 berarti model GloVe yang dilatih pada Wikipedia dan Gigaword, dengan vektor berukuran 50 dimensi. Ada banyak model lain yang bisa dipilih. Jika model belum ada di komputer Anda, ia akan diunduh.

3. vector_kucing = model_glove[kata_kucing]: Setelah model dimuat, Anda bisa mengakses vektor representasi untuk sebuah kata hanya dengan menggunakan kata tersebut sebagai kunci indeks (seperti kamus Python).

4. model_glove.most_similar(kata_target): Fungsi ini mencari kata-kata lain dalam kamus model yang memiliki vektor paling "dekat" (paling mirip) dengan vektor kata_target. Ini menunjukkan konsep bahwa kata-kata dengan makna serupa akan berdekatan di ruang vektor.

5. model_glove.most_similar(positive=['king', 'woman'], negative=['man']): Ini adalah demonstrasi yang menarik dari sifat aljabar linear pada Word Embeddings. Kita mencari kata yang paling mirip dengan king dan woman, setelah "mengurangi" (negative) man. Secara intuitif, ini mencari V_king−V_man+V_woman. Hasil idealnya adalah queen. Ini menunjukkan bagaimana Word Embeddings menangkap hubungan semantik.

#Melatih Model Word2Vec Sendiri

Jika Anda ingin melatih model Word2Vec Anda sendiri dari korpus yang lebih kecil, gensim juga menyediakannya. Ini bagus untuk memahami bagaimana prosesnya bekerja.

In [ ]:
from gensim.models import Word2Vec

# Contoh korpus (daftar kalimat yang sudah ditokenisasi)
# Setiap "kalimat" adalah daftar kata-kata. Ini adalah hasil dari proses tokenisasi Anda.
korpus_tokenized = [
    ['saya', 'suka', 'makan', 'apel'],
    ['saya', 'suka', 'pisang', 'dan', 'apel'],
    ['kucing', 'suka', 'ikan'],
    ['anjing', 'suka', 'bermain'],
    ['raja', 'adalah', 'pria', 'penguasa'],
    ['ratu', 'adalah', 'wanita', 'penguasa'],
    ['pria', 'bekerja'],
    ['wanita', 'belajar']
]

In [ ]:
print("\n--- Melatih Model Word2Vec Sendiri ---")

# Inisialisasi dan latih model Word2Vec
# parameters:
#   sentences: data pelatihan (daftar daftar kata)
#   vector_size: jumlah dimensi vektor (misal: 100 dimensi)
#   window: ukuran jendela konteks (berapa banyak kata di sekitar yang dilihat)
#   min_count: mengabaikan kata yang frekuensinya di bawah ambang batas ini
#   workers: jumlah thread CPU untuk pelatihan paralel
model_saya = Word2Vec(sentences=korpus_tokenized, vector_size=10, window=2, min_count=1, workers=4)

In [ ]:
# Akses vektor kata
kata_apel = "apel"
if kata_apel in model_saya.wv: # .wv singkatan dari word vectors
    print(f"\nVektor untuk '{kata_apel}': {model_saya.wv[kata_apel]}")
else:
    print(f"Kata '{kata_apel}' tidak ditemukan dalam kamus model saya.")

In [ ]:
# Mencari kata-kata paling mirip
kata_target_saya = "raja"
print(f"\nKata-kata paling mirip dengan '{kata_target_saya}':")
if kata_target_saya in model_saya.wv:
    similar_words_saya = model_saya.wv.most_similar(kata_target_saya)
    for word, similarity in similar_words_saya:
        print(f"- {word}: {similarity:.4f}")
else:
    print(f"Kata '{kata_target_saya}' tidak ditemukan dalam kamus model saya.")

In [ ]:
# Analogi (mungkin tidak terlalu akurat karena korpus kecil)
print("\n--- Analogi Kata (Model Saya) ---")
positive_saya = ['raja', 'wanita']
negative_saya = ['pria']
try:
    result_saya = model_saya.wv.most_similar(positive=positive_saya, negative=negative_saya)
    for word, similarity in result_saya:
        print(f"- {word}: {similarity:.4f}")
except KeyError as e:
    print(f"Salah satu kata untuk analogi tidak ditemukan: {e}")

Penjelasan Kode Melatih Model Word2Vec Sendiri:
1. from gensim.models import Word2Vec: Mengimpor kelas Word2Vec dari gensim.

2. korpus_tokenized = [...]: Ini adalah input untuk pelatihan. Ingat, Word2Vec membutuhkan daftar kalimat, di mana setiap kalimat adalah daftar token (kata). Ini adalah hasil dari langkah tokenisasi dan pra-pemrosesan Anda.

3. model_saya = Word2Vec(...): Membuat dan melatih model Word2Vec.

 - sentences: Data teks yang akan digunakan untuk pelatihan.

 - vector_size: Ukuran dimensi vektor yang akan dihasilkan untuk setiap kata (misalnya, 100). Semakin besar, semakin banyak informasi yang bisa ditampung, tapi butuh lebih banyak data dan komputasi.

 - window: Jendela konteks. Ini adalah jumlah kata di sebelah kiri dan kanan dari kata target yang akan dipertimbangkan sebagai konteks.

 - min_count: Jika sebuah kata muncul kurang dari min_count kali di seluruh korpus, kata tersebut akan diabaikan. Ini membantu menyingkirkan kata-kata yang sangat jarang dan mengurangi ukuran kamus.

 - workers: Jumlah core CPU yang akan digunakan untuk pelatihan paralel.

4. model_saya.wv[kata_apel]: .wv adalah objek yang menyimpan semua vektor kata yang sudah dilatih. Anda bisa mengakses vektor kata melalui objek ini.

5. model_saya.wv.most_similar(...): Sama seperti model pre-trained, Anda bisa mencari kata-kata yang paling mirip dan melakukan operasi analogi. Catatan: Untuk korpus sekecil ini, hasilnya mungkin tidak sempurna.